In [1]:
import tensorflow.python.keras as k
k.backend.set_floatx("posit160")

import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import neural_network
from sklearn import  metrics
import matplotlib.pyplot as plt

2023-06-21 11:08:08.401396: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/count
2023-06-21 11:08:08.401455: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/count
2023-06-21 11:08:08.401461: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/write/api
2023-06-21 11:08:08.401463: E tensorflow/core/lib/monitoring/collection_registry.cc:77] Cannot register 2 metrics with the same name: /tensorflow/core/saved_model/read/api


In [2]:
def plot(history):
  acc = history.history['accuracy']
  loss = history.history['loss']
  epochs = range(len(acc))
  plt.figure()
  plt.plot(epochs, acc, 'bo', label='Training acc')
  plt.title('Training accuracy')
  plt.legend()
  #plt.savefig('ac.png')
  plt.figure()
  plt.plot(epochs, loss, 'bo', label='Training loss')
  plt.title('Training loss')
  plt.legend()
  #plt.savefig('loss.png')
  plt.show()

In [3]:
train = pd.read_csv("/home/fabiano/cppWorkspace/examples/train-mnist.csv")

Y = train['label'][:2000] # use more number of rows for more training 
X = train.drop(['label'], axis = 1)[:2000] # use more number of rows for more training 
x_train, x_val, y_train, y_val = train_test_split(X.values, Y.values, test_size=0.10, random_state=42)

# network parameters 
batch_size = 128
num_classes = 10
epochs = 5 # Further Fine Tuning can be done

# input image dimensions
img_rows, img_cols = 28, 28

# preprocess the train data 
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_train /= 255

# preprocess the validation data
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
x_val = x_val.astype('float32')
x_val /= 255

input_shape = (img_rows, img_cols, 1)

# convert the target variable 
y_train = k.utils.np_utils.to_categorical(y_train, num_classes)
y_val = k.utils.np_utils.to_categorical(y_val, num_classes)

In [4]:
model = k.models.Sequential()
#model.add(k.layers.Conv2D(16, kernel_size=(3, 3), activation='tanh', input_shape=input_shape))
model.add(k.layers.Flatten(input_shape=input_shape))
model.add(k.layers.Dense(128, activation='tanh'))
#model.add(k.layers.BatchNormalization())
#model.add(k.layers.Dropout(0.05))
model.add(k.layers.Dense(num_classes, activation='softmax'))

model.compile(loss=k.losses.categorical_crossentropy,  optimizer='adam', metrics=['accuracy'])

earlystop_callback = k.callbacks.EarlyStopping(monitor='val_accuracy',patience=25)

2023-06-21 11:08:19.794571: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data=(x_val, y_val), callbacks=earlystop_callback)

2023-06-21 11:08:19.986849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:164] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50
15/15 [==============================] - 1s 74ms/step - loss: 2.2930 - accuracy: 0.1133 - val_loss: 2.3125 - val_accuracy: 0.0750
Epoch 2/50
15/15 [==============================] - 1s 66ms/step - loss: 2.2930 - accuracy: 0.1028 - val_loss: 2.3193 - val_accuracy: 0.0750
Epoch 3/50
15/15 [==============================] - 1s 60ms/step - loss: 2.3018 - accuracy: 0.1033 - val_loss: 2.2920 - val_accuracy: 0.0950
Epoch 4/50
15/15 [==============================] - 1s 58ms/step - loss: 2.2930 - accuracy: 0.0911 - val_loss: 2.3174 - val_accuracy: 0.0750
Epoch 5/50
15/15 [==============================] - 1s 59ms/step - loss: 2.3105 - accuracy: 0.1111 - val_loss: 2.3096 - val_accuracy: 0.1349
Epoch 6/50
15/15 [==============================] - 1s 67ms/step - loss: 2.3018 - accuracy: 0.0994 - val_loss: 2.2969 - val_accuracy: 0.1000
Epoch 7/50
15/15 [==============================] - 1s 87ms/step - loss: 2.2930 - accuracy: 0.1472 - val_loss: 2.2998 - val_accuracy: 0.2100
Epoch 8/50
15

In [6]:
prediction = model.predict(x_train[:1])
print("prediction shape:", prediction.shape)
prediction

prediction shape: (1, 10)


array([[0.00276566, 5.14984e-05, 0.00258636, 0.979736, 9.39369e-05,
        0.00912476, 0.000198841, 0.00166893, 0.00370407, 0.00112343]],
      dtype=posit160)

In [7]:
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases  = model.layers[1].get_weights()[1]
first_layer_weights

array([[-0.0801697, -0.0809021, -0.0800171, ..., -0.0805664, -0.0803528,
        -0.0799866],
       [-0.080719, -0.0809631, -0.0808105, ..., -0.0802307, -0.0801086,
        -0.0805359],
       [-0.0810242, -0.0802917, -0.0805664, ..., -0.0801697, -0.080658,
        -0.0808716],
       ...,
       [-0.0809326, -0.0803528, -0.08078, ..., -0.0810547, -0.0805969,
        -0.0801086],
       [-0.0809631, -0.0807495, -0.0809631, ..., -0.0803223, -0.0800781,
        -0.0806885],
       [-0.0808105, -0.0803833, -0.0800171, ..., -0.0804138, -0.0805359,
        -0.0809631]], dtype=posit160)